In [51]:
import pickle
from spkit import bkt
import numpy as np
import pandas as pd
from sklearn.metrics import roc_curve, auc

In [2]:
with open('kdd_2005_2005_df.pkl', 'rb') as pklfile:
    df = pickle.load(pklfile)
with open('kdd_2005_2005_q_matrix.pkl', 'rb') as pklfile:
    q_matrix = pickle.load(pklfile)
with open('kdd_2005_2005_students.pkl', 'rb') as pklfile:
    students = pickle.load(pklfile)

In [3]:
%%time
# Data matrix
data = df[['Correct First Attempt', 'Anon Student Id', 'question_id']].values.tolist()

# Instantiate model
model = bkt.BKT()
model.fit(data, q_matrix)

CPU times: user 8.84 s, sys: 126 ms, total: 8.97 s
Wall time: 12.8 s


In [60]:
%%time
data_arr = np.asarray(data)
rmse = 0
n = 0
score_df = []
for student in students:
    student_idx = np.where(data_arr[:,1] == student)
    outcome = data_arr[student_idx, 0].astype('int64')
    question_id = data_arr[student_idx, 2].astype('int64')
    data_student = np.vstack((outcome, question_id)).T.astype('int64').tolist()
    predicted = model.predict_proba(data_student, q_matrix)
    s_aic, s_bic, s_rmse, s_acc = model.score()
    fpr, tpr, _ = roc_curve(outcome, predicted[:,0])
    roc_auc_correct = auc(fpr, tpr)
    fpr, tpr, _ = roc_curve(outcome, predicted[:,1])
    roc_auc_incorrect = auc(fpr, tpr)
    score_df.append({"student": student,
                     "AIC": s_aic,
                     "BIC": s_bic,
                     "RMSE": s_rmse,
                     "Acc": s_acc,
                     "n questions": model.n_questions,
                     "LL": model.loglikelihood.sum(),
                     "correct_outcome": np.where(outcome == '1')[1].shape[0],
                     "incorrect_outcome": np.where(outcome == '0')[1].shape[0],
                     "AUC correct": roc_auc_correct,
                     "AUC incorrect": roc_auc_incorrect})

ValueError: multilabel-indicator format is not supported

In [59]:
predicted[:,0].shape

(587,)

In [39]:
score = pd.DataFrame.from_dict(score_df)

In [20]:
rmse_train = np.sqrt((score['RMSE']**2*score['n questions']).sum()/score['n questions'].sum())
rmse_avg = score['RMSE'].mean()
rmse_std = score['RMSE'].std()

In [22]:
print("RMSE de treino: %.2f" % rmse_train)
print("RMSE médio: %.2f +- %.2f" % (rmse_avg, rmse_std))

RMSE de treino: 0.42
RMSE médio: 0.43 +- 0.05


In [40]:
acc_train = (score['Acc']*score['n questions']).sum()/score['n questions'].sum()
acc_avg = score['Acc'].mean()
acc_std = score['Acc'].std()

In [41]:
print("Acc de treino: %.2f" % acc_train)
print("Acc médio: %.2f +- %.2f" % (acc_avg, acc_std))

Acc de treino: 0.75
Acc médio: 0.72 +- 0.11


In [46]:
score['desbalanceamento'] = score['correct_outcome']/score['n questions']

In [47]:
score[['Acc', 'desbalanceamento']]

,Acc,desbalanceamento
0,0.693356,0.732538
1,0.583333,0.666667
2,0.677078,0.684767
3,0.796472,0.802926
4,0.715473,0.723785
5,0.758889,0.763969
6,0.759542,0.782443
7,0.796412,0.812012
8,0.793664,0.800569
9,0.709909,0.724902
